In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

In [14]:
# Setup
options = Options()
options.add_experimental_option("detach", True)

chrome_options = webdriver.ChromeOptions()
# Disable "Save As" dialog and set default download directory
download_dir = 'C:/Users/dloso/Downloads'
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
url = 'https://www.wunderground.com/history/monthly/KPDX/date/2025-9' # Replace with the actual page URL

driver.get(url)

df = pd.read_html(driver.page_source)[1]
df.head()

C:\Users\dloso\AppData\Local\Temp\ipykernel_27556\890032164.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(driver.page_source)[1]


,Time,Temperature (°F),Dew Point (°F),Humidity (%),Wind Speed (mph),Pressure (in),Precipitation (in)
0,Sep 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...,Max Avg Min 78 67.6 62 89 72.2 61 83 74.7 67 8...,Max Avg Min 59 56.9 56 64 58.5 56 65 61.9 60 6...,Max Avg Min 84 69.7 46 87 64.8 36 81 65.2 49 8...,Max Avg Min 9 5.7 3 9 4.0 0 9 4.3 0 10 5.3 0 1...,Max Avg Min 30.0 30.0 30.0 30.0 30.0 29.9 30.0...,Total 0.00 0.00 0.00 0.00 0.00 0.04 0.00 0.00 ...
1,Sep,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,NaN,NaN,NaN,NaN
4,3,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.columns = [
    f"{a}_{b}" if "Unnamed" not in b and "nan" not in str(b) else a 
    for a, b in df.columns.to_flat_index()
]

ValueError: too many values to unpack (expected 2)

In [6]:
# Example post-processing to flatten the MultiIndex columns
# 1. Get the top level of the columns
top_level = df.columns.get_level_values(0).to_series()
top_level


Time                                Time
Temperature (°F)        Temperature (°F)
Dew Point (°F)            Dew Point (°F)
Humidity (%)                Humidity (%)
Wind Speed (mph)        Wind Speed (mph)
Pressure (in)              Pressure (in)
Precipitation (in)    Precipitation (in)
dtype: object

In [7]:
# 2. Forward-fill the 'Unnamed' values in the top level
# The mask() function helps identify and fill 'Unnamed' or similar default names
cleaned_top_level = top_level.mask(top_level.str.startswith('Unnamed')).ffill().fillna('')

# 3. Get the bottom level of the columns
bottom_level = df.columns.get_level_values(1)

# 4. Combine the levels into a single list of strings
df.columns = [f'{a} {b}'.strip() for a, b in zip(cleaned_top_level, bottom_level)]

# Now df has single-level, clean column headers

IndexError: Too many levels: Index has only 1 level, not 2

In [ ]:
busiest_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'JFK', 'CLT', 'LAS', 'MCO',
                    'MIA', 'PHX', 'SEA', 'SFO', 'EWR', 'IAH', 'BOS', 'MSP', 'FLL',
                    'LGA', 'DTW', 'PHL', 'SLC', 'BWI', 'IAD', 'SAN', 'DCA', 'TPA',
                    'BNA', 'AUS', 'HNL', 'MDW', 'DAL', 'PDX']
month_days = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30,
                              'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
for airport in busiest_airports:
    df = pd.DataFrame()
    
    for cur_year in range(2021, 2026):
        if cur_year == 2020 or cur_year == 2024:
            month_days['Feb'] = 29
        else:
            month_days['Feb'] = 28
            
        for cur_month in range(1, 13):
            url = 'https://www.wunderground.com/history/monthly/' + 'K' + airport + '/date/' + str(cur_year) + '-' + str(cur_month)
            driver = webdriver.Chrome()
            driver.get(url)
            
            try:
                # 1. Wait for the specific observation table container to load
                wait = WebDriverWait(driver, 2)
                table_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "observation-table")))
            
                # 2. Extract headers (Top Level and Subheaders)
                # WU often uses 'thead' or specific 'tr' classes for these
                header_rows = table_container.find_elements(By.TAG_NAME, "tr")[:2]
                
                # Extract all data rows
                data_rows = table_container.find_elements(By.TAG_NAME, "tr")[2:]

                # String out data into a list of lists
                all_data = []
                for row in data_rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    all_data.append([c.text for c in cols])
            
                data_dict = {}
                days_in_month = month_days[list(month_days.keys())[cur_month-1]]
                data_dict[all_data[0][0]] = [date[0] for date in all_data[1:1+days_in_month]]
                data_dict['Temp_' + all_data[1+days_in_month][0]] = [temp[0] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['Temp_' + all_data[1+days_in_month][1]] = [temp[1] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['Temp_' + all_data[1+days_in_month][2]] = [temp[2] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][0]] = [windspeed[0] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][1]] = [windspeed[1] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][2]] = [windspeed[2] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][0]] = [pressure[0] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][1]] = [pressure[1] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][2]] = [pressure[2] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Precipitation' + all_data[6+days_in_month*6][0]] = [precip[0] for precip in all_data[7+days_in_month*6:7+days_in_month*7]]
                
                df = pd.concat([df, pd.DataFrame(data_dict)])
            finally:
                driver.quit()
                print(f'{airport} {list(month_days.keys())[cur_month-1]} {cur_year} complete.')

    df.to_csv('data/weather/' + airport + '_Weather.csv')

ATL Jan 2021 complete.


In [51]:
data_dict = {}
# count number of days in the month
day_count = 0
for item in all_data[1:]:
    try:
        # Attempt to convert to float
        float(item)
        day_count += 1
    except ValueError:
        # Stop counting once we hit a non-numeric string (the end string)
        break
data_dict[all_data[0][0]] = [date[0] for date in all_data[1:31]]
data_dict['Temp_' + all_data[31][0]] = [temp[0] for temp in all_data[32:62]]
data_dict['Temp_' + all_data[31][1]] = [temp[1] for temp in all_data[32:62]]
data_dict['Temp_' + all_data[31][2]] = [temp[2] for temp in all_data[32:62]]
data_dict['WindSpeed_' + all_data[124][0]] = [windspeed[0] for windspeed in all_data[125:155]]
data_dict['WindSpeed_' + all_data[124][1]] = [windspeed[1] for windspeed in all_data[125:155]]
data_dict['WindSpeed_' + all_data[124][2]] = [windspeed[2] for windspeed in all_data[125:155]]
data_dict['Pressure_' + all_data[155][0]] = [pressure[0] for pressure in all_data[156:186]]
data_dict['Pressure_' + all_data[155][1]] = [pressure[1] for pressure in all_data[156:186]]
data_dict['Pressure_' + all_data[155][2]] = [pressure[2] for pressure in all_data[156:186]]
data_dict['Precipitation' + all_data[186][0]] = [precip[0] for precip in all_data[187:217]]

In [52]:
df = pd.DataFrame(data_dict)

In [53]:
df.head()

,Sep,Temp_Max,Temp_Avg,Temp_Min,WindSpeed_Max,WindSpeed_Avg,WindSpeed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,PrecipitationTotal
0,1,78,67.6,62,9,5.7,3,30.0,30.0,30.0,0.00
1,2,89,72.2,61,9,4.0,0,30.0,30.0,29.9,0.00
2,3,83,74.7,67,9,4.3,0,30.0,30.0,29.9,0.00
3,4,87,73.8,65,10,5.3,0,29.9,29.9,29.8,0.00
4,5,76,67.8,63,12,5.1,0,29.9,29.8,29.7,0.00


In [56]:
result = {}
current_key = None

for sublist in all_data:
    for item in sublist:
        try:
            # Try to convert to float
            value = float(item)
            if current_key is not None:
                result[current_key].append(value)
        except ValueError:
            # If conversion fails, it's a string key
            current_key = item
            result.setdefault(current_key, [])

result

{'Sep': [1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0],
 'Max': [],
 'Avg': [],
 'Min': [78.0,
  67.6,
  62.0,
  89.0,
  72.2,
  61.0,
  83.0,
  74.7,
  67.0,
  87.0,
  73.8,
  65.0,
  76.0,
  67.8,
  63.0,
  73.0,
  66.7,
  62.0,
  75.0,
  66.3,
  62.0,
  78.0,
  67.2,
  57.0,
  79.0,
  69.7,
  64.0,
  73.0,
  66.4,
  62.0,
  68.0,
  63.9,
  62.0,
  73.0,
  64.8,
  60.0,
  82.0,
  66.2,
  58.0,
  70.0,
  63.0,
  58.0,
  76.0,
  60.3,
  49.0,
  88.0,
  73.3,
  56.0,
  82.0,
  69.5,
  59.0,
  78.0,
  67.0,
  59.0,
  82.0,
  67.2,
  55.0,
  81.0,
  66.7,
  56.0,
  71.0,
  61.3,
  55.0,
  77.0,
  61.8,
  51.0,
  88.0,
  69.1,
  53.0,
  83.0,
  69.7,
  56.0,
  73.0,
  62.0,
  54.0,
  73.0,
  60.0,
  49.0,
  80.0,
  65.0,
  51.0,
  76.0,
  64.9,
  55.0,
  64.0,
  61.9,
  59.0,
  72.0,
  60.3,
  55.0,
  59.0,
  